In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from obspy.geodetics import gps2dist_azimuth

project = 'july6'
traindate = '2023-07-27'
testdate = '2023-07-28'
traindate_path = '/home/sdybing/gnss-picker/cnn_models_outputs/' + project + '_fq_train/models/traindate_' + traindate + '/'
test_outputs_path = traindate_path + 'data/'
figure_save_dir = traindate_path + 'figures/'

results = np.load(test_outputs_path + 'fqtest_metadata_with_results_pgd_snr.npy')
# New metadata array columns:

# 0: FQ rupture name
# 1: station name
# 2: magnitude
# 3: result (true pos, etc.)
# 4: PGD
# 5: SNR_N
# 6: SNR_E
# 7: SNR_Z

waveforms = np.load(test_outputs_path + testdate + '_fqtest_data.npy')


In [2]:
eq_lats = []
eq_lons = []
eq_depths = []
sta_lats = []
sta_lons = []
dist_ms = []

for i in range(len(results)):
    
    if results[i][0] == 'nan':
        rupture = 'nan'
        station = 'nan'
        mag = 'nan'
        pgd = 'nan'
        
        if results[i][3] == 'true pos': # predicted 1, target 1
            result = 'true_pos'
        
        elif results[i][3] == 'true neg': # predicted 0, target 0
            result = 'true_neg'
            
        elif results[i][3] == 'false pos': # predicted 1, target 0
            result = 'false_pos'
        
        elif results[i][3] == 'false neg': # predicted 0, target 1
            result = 'false_neg'
        
        eq_lat = 'nan'
        eq_lon = 'nan'
        eq_depth = 'nan'
        sta_lat = 'nan'
        sta_lon = 'nan'
        dist_m = 'nan'
        
        eq_lats.append(eq_lat)
        eq_lons.append(eq_lon)
        eq_depths.append(eq_depth)
        sta_lats.append(sta_lat)
        sta_lons.append(sta_lon)
        dist_ms.append(dist_m)
    
    else:
        
        # Getting rupture name and calculating hypocentral distance
        
        rupture = results[i][0]

        log = glob('/hdd/rc_fq/summer23/july6/output/ruptures/' + rupture + '.log') # for Valdivia
    
        f = open(log[0],'r')
        line = f.readlines()
        
        # Getting hypocenter location
        hyp_loc_junk = line[16]
        eq_lon = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[0])
        eq_lat = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[1])
        eq_depth = float(hyp_loc_junk.split(' ')[2].split('(')[1].split(')')[0].split(',')[2])
        eq_depth_m = eq_depth * 1000
        
        station = results[i][1]
        
        # Getting PGD from the waveforms themselves
        
        n_data = waveforms[i,:,0]
        e_data = waveforms[i,:,1]
        z_data = waveforms[i,:,2]
        pgd = np.max(np.sqrt((n_data)**2+(e_data)**2+(z_data)**2))
        
        station_names = np.genfromtxt('/hdd/rc_fq/summer23/july6/data/station_info/rc_gflist.gflist', dtype = 'U', usecols = 0)
        
        k = np.where(station_names == station)[0]
        
        # Getting station location
        
        station_info = np.genfromtxt('/hdd/rc_fq/summer23/july6/data/station_info/rc_gflist.gflist', dtype = 'U')
        sta_lon = float(station_info[0][1])
        sta_lat = float(station_info[0][2])

        # Calculating hypocentral distance

        distaz = gps2dist_azimuth(eq_lat, eq_lon, sta_lat, sta_lon)
        dist_m = distaz[0]
        
        # Adding info about result of training
        
        if results[i][3] == 'true pos': # predicted 1, target 1
            mag = float(results[i][2])
            dot = 2
            result = 'true_pos'
        
        elif results[i][3] == 'true neg': # predicted 0, target 0
            mag = float(results[i][2])
            dot = 1
            result = 'true_neg'
            
        elif results[i][3] == 'false pos': # predicted 1, target 0
            mag = float(results[i][2])
            dot = 0
            result = 'false_pos'
        
        elif results[i][3] == 'false neg': # predicted 0, target 1
            mag = float(results[i][2])
            dot = -1
            result = 'false_neg'
            
        else:
            pass

        eq_lats.append(eq_lat)
        eq_lons.append(eq_lon)
        eq_depths.append(eq_depth)
        sta_lats.append(sta_lat)
        sta_lons.append(sta_lon)
        dist_ms.append(dist_m)


In [3]:
# Add to the results metadata array

eqlats_vector = np.array(eq_lats).reshape(len(eq_lats),1) 
eqlons_vector = np.array(eq_lons).reshape(len(eq_lons),1) 
eqdepths_vector = np.array(eq_depths).reshape(len(eq_depths),1) 
stalats_vector = np.array(sta_lats).reshape(len(sta_lats),1) 
stalons_vector = np.array(sta_lons).reshape(len(sta_lons),1) 
distms_vector = np.array(dist_ms).reshape(len(dist_ms),1) 

a = np.append(results, eqlats_vector, axis = 1) 
b = np.append(a, eqlons_vector, axis = 1)
c = np.append(b, eqdepths_vector, axis = 1)
d = np.append(c, stalats_vector, axis = 1)
e = np.append(d, stalons_vector, axis = 1)
new_meta_array = np.append(e, distms_vector, axis = 1)

# New metadata array columns:

# 0: FQ rupture name
# 1: station name
# 2: magnitude
# 3: result (true pos, etc.)
# 4: PGD
# 5: SNR_N
# 6: SNR_E
# 7: SNR_Z
# 8: FQ rupture hypocenter lat
# 9: FQ rupture hypocenter lon
# 10: FQ rupture hypocenter depth
# 11: station lat
# 12: station lon
# 13: hypocentral distance (m)

np.save(test_outputs_path + 'fqtest_metadata_with_results_pgd_snr_locs_hypdists.npy', new_meta_array)

In [4]:
new_meta_array.shape

(91738, 14)